In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.0


In [2]:
#iris 데이터셋 활용
iris = load_iris()

#데이터 확인
print(iris.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [3]:
# input, output 분리
X = iris['data']
y = iris['target']

print(X.shape)
print(y.shape)

# 개발/검증 분리
# stratify : y 라벨 비중 유지
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

print(trainX.shape)
print(testX.shape)

(150, 4)
(150,)
(120, 4)
(30, 4)


In [5]:
# 빈 모델을 선언
model = tf.keras.models.Sequential()

# 빈 모델에 첫번째 layer 추가
# units에는 hidden node 갯수를 명시하고, input 차원에 train 데이터의 차원을 입력
model.add(tf.keras.layers.Dense(units=64, input_dim=trainX.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(units=128, input_dim=trainX.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(units=128, input_dim=trainX.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(units=64, input_dim=trainX.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(units=64, input_dim=trainX.shape[1], activation='relu'))
model.add(tf.keras.layers.BatchNormalization(
    momentum = 0.95,
    epsilon = 0.005,
    beta_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev = 0.05),
    gamma_initializer = tf.keras.initializers.Constant(value=0.9)
))


model.add(tf.keras.layers.Dense(units=3, activation='softmax'))

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer = optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# target을 범주화
trainY_onehot = tf.keras.utils.to_categorical(trainY)
history = model.fit(trainX, trainY_onehot, batch_size=16, epochs=30)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8

In [6]:
testY_onehot = tf.keras.utils.to_categorical(testY)
results = model.evaluate(testX, testY_onehot)

1/1 [==============================] - 0s 233ms/step - loss: 0.2098 - accuracy: 0.8667
